In [13]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [14]:
shortage_tracts_gdf = gpd.read_file('./data/shortage_tracts_gdf_1pcp220.geojson')
counties_gdf = gpd.read_file('./data/32counties.geojson')
counties_gdf = counties_gdf.to_crs(shortage_tracts_gdf.crs)
shortage_tracts_gdf["COUNTYFP"] = "36" + shortage_tracts_gdf["COUNTYFP"]

In [15]:
# Step 1: Group by 'COUNTYFP' and sum multiple columns
columns_to_sum = ['unserved_medicaid', 'unserved_commercial']
summed_data = shortage_tracts_gdf.groupby('COUNTYFP')[columns_to_sum].sum().reset_index()

# Step 2: Merge the summed data with 'counties_gdf'
merged_gdf = counties_gdf.merge(summed_data, how='left', left_on='FIPS_CODE', right_on='COUNTYFP')
merged_gdf = merged_gdf[[
    'NAME', 'Shape_Area', 'geometry', 'FIPS_CODE', 'unserved_medicaid', 'unserved_commercial'
]]
merged_gdf.rename(columns={'FIPS_CODE': 'id'}, inplace=True)


# Caclculate for unserved population / km2
merged_gdf['unserved medicaid enrollees / km2'] = (merged_gdf['unserved_medicaid'] / merged_gdf['Shape_Area'] * 1000000).round(2)
merged_gdf['unserved commercial enrollees / km2'] = (merged_gdf['unserved_commercial'] / merged_gdf['Shape_Area'] * 1000000).round(2)
merged_gdf['unserved population / km2'] = merged_gdf['unserved medicaid enrollees / km2'] + merged_gdf['unserved commercial enrollees / km2']

In [16]:
# Reproject the GeoDataFrame to EPSG:4326
merged_gdf = merged_gdf.to_crs(epsg=4326)

# Replace NaN with 0
merged_gdf = merged_gdf.fillna(0)

In [17]:
merged_gdf.to_file('shortage_counties.geojson', driver='GeoJSON')

In [19]:
# Get list of {min:x, max:y, name:z} object
columns = [
    "unserved medicaid enrollees / km2", 
    "unserved commercial enrollees / km2", 
    "unserved population / km2"
]

# Generate the list of dictionaries
stats_list = [
    {
        'name': col,
        'min': merged_gdf[col].min(),
        'max': merged_gdf[col].max()
    } 
    for col in columns
]

# Display the resulting list of dictionaries
print(stats_list)

[{'name': 'unserved medicaid enrollees / km2', 'min': 0.0, 'max': 1785.49}, {'name': 'unserved commercial enrollees / km2', 'min': 0.0, 'max': 461.61}, {'name': 'unserved population / km2', 'min': 0.0, 'max': 2220.16}]


In [21]:
import json
json_data = json.dumps(stats_list)

# Set the filename where you want to store the JSON data
filename = '32counties_min_max.json'

# Write the JSON string to a file
with open(filename, 'w') as file:
    file.write(json_data)

print(f"JSON data has been written to {filename}")

JSON data has been written to 32counties_min_max.json
